## Problem

CCC動物園ではコアラを飼育していますが、近頃ぐうたらし過ぎて太ってしまいました。そこで、ダイエットのために糖質制限をかけて餌を与えることにします。主食のユーカリには糖質の他にも食物繊維、ミネラル、毒素が含まれていますが、ミネラル以外のものは取りすぎるとよくないので、こちらも摂取制限をかけます。
成分構成の異なる二つのユーカリをうまく食べ合わせて、なるべく多くのミネラルを摂取するにはどうすればいいでしょうか。
Q
|     成分 | ユーカリA | ユーカリB | 摂取上限 |
|       -  |         - |         - |        - |
|     糖質 |        10 |      15   |     6000 |
| 食物繊維 |        15 |      10   |     6000 |
|     毒素 |       2.5 |       0.5 |      900 |
| ミネラル |       0.4 |       0.5 |   最大化 |

(https://www.chuo-computer.co.jp/archives/12101)

In [1]:
import pulp
import pandas as pd

In [2]:
df = pd.DataFrame(
columns=["ユーカリA", "ユーカリB", "接種上限"],
index=["糖質", "食物繊維", "毒素", "ミネラル"],
data=[
    [10, 15, 6000],
    [15, 10, 6000],
    [2.5, 0.5, 900],
    [0.4, 0.5, "最大化"],
     ]
)
df

,ユーカリA,ユーカリB,接種上限
糖質,10.0,15.0,6000
食物繊維,15.0,10.0,6000
毒素,2.5,0.5,900
ミネラル,0.4,0.5,最大化


In [3]:
problem = pulp.LpProblem(name="diet", sense=pulp.LpMaximize)
problem

diet:
MAXIMIZE
None
VARIABLES

In [4]:
foods = df.columns[:-1].values.tolist()
components = df.index[:].values.tolist()
print(foods, components)

['ユーカリA', 'ユーカリB'] ['糖質', '食物繊維', '毒素', 'ミネラル']


In [5]:
table = pulp.LpVariable.dict("", (foods), lowBound=0, cat=pulp.LpContinuous)
table

{'ユーカリA': _ユーカリA, 'ユーカリB': _ユーカリB}

In [6]:
problem += pulp.lpSum([table[f] * df[f]["ミネラル"] for f in table.keys()])
problem

diet:
MAXIMIZE
0.4*_ユーカリA + 0.5*_ユーカリB + 0.0
VARIABLES
_ユーカリA Continuous
_ユーカリB Continuous

In [7]:
for c in components[:-1]:
    problem += pulp.lpSum([table[f] * df[f][c] for f in table.keys()]) <= df["接種上限"][c]
problem

diet:
MAXIMIZE
0.4*_ユーカリA + 0.5*_ユーカリB + 0.0
SUBJECT TO
_C1: 10 _ユーカリA + 15 _ユーカリB <= 6000

_C2: 15 _ユーカリA + 10 _ユーカリB <= 6000

_C3: 2.5 _ユーカリA + 0.5 _ユーカリB <= 900

VARIABLES
_ユーカリA Continuous
_ユーカリB Continuous

In [8]:
status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
print("Status", pulp.LpStatus[status])

Status Optimal


In [9]:
print("=" * 3, "Result", "=" * 29)
print('ミネラル: {}'.format(pulp.value(problem.objective)))
print("-" * 40)
for f in foods:
    print(table[f].value())
print("=" * 40)

=== Result =============================
ミネラル: 216.0
----------------------------------------
240.0
240.0
